# Database Demo

Sample functionality for creating tables, inserting data and running similarity search with OgbujiPT.

Notes:
- `pip install jupyter` if notebook is not running

This notebook will attempt to access a database named `PGv` at `sofola:5432`, using the username `oori` and password `example`. If you have a different setup, you can change the connection string in the first cell.

## Initial setup and Imports

In [38]:
DB_NAME = 'PGv'
HOST = 'sofola'
PORT = 5432
USER = 'oori'
PASSWORD = 'example'

In [39]:
import json
import uuid

from ogbujipt.embedding.pgvector import PGvectorHelper

from sentence_transformers     import SentenceTransformer

e_model = SentenceTransformer('all-MiniLM-L6-v2')  # Load the embedding model

pacer_copypasta = [  # Demo data
    'The FitnessGram™ Pacer Test is a multistage aerobic capacity test that progressively gets more difficult as it continues.', 
    'The 20 meter pacer test will begin in 30 seconds. Line up at the start.', 
    'The running speed starts slowly, but gets faster each minute after you hear this signal.', 
    '[beep] A single lap should be completed each time you hear this sound.', 
    '[ding] Remember to run in a straight line, and run as long as possible.', 
    'The second time you fail to complete a lap before the sound, your test is over.', 
    'The test will begin on the word start. On your mark, get ready, start.'
]


abbott_and_costello = [
    {'role': 'system', 'content': 'The user is considering becoming a ballplayer. The assistant wants to make sure he knows what he\'s getting into.'},
    {'role': 'assistant', 'content': 'Strange as it may seem, they give ball players nowadays very peculiar names.'},
    {'role': 'user', 'content': 'Funny names?'},
    {'role': 'assistant', 'content': 'Nicknames, nicknames. Now, on the St. Louis team we have Who\'s on first, What\'s on second, I Don\'t Know is on third--'},
    {'role': 'user', 'content': 'That\'s what I want to find out. I want you to tell me the names of the fellows on the St. Louis team.'},
    {'role': 'assistant', 'content': "I'm telling you. Who is on first. What's on second. I Don't Know's on third--"},
    {'role': 'user', 'content': "You know the fellows' names?"},
    {'role': 'assistant', 'content': 'Yes.'},
    {'role': 'user', 'content': "Well, then who's playing first?"},
    {'role': 'assistant', 'content': 'Yes.'},
    {'role': 'user', 'content': "I mean the fellow's name on first base."},
    {'role': 'assistant', 'content': 'Who.'},
    {'role': 'user', 'content': "The fellow playin' first base."},
    {'role': 'assistant', 'content': 'Who.'},
    {'role': 'user', 'content': "The guy on first base."},
    {'role': 'assistant', 'content': 'Who is on first.'},
    {'role': 'user', 'content': "Well, what are you askin' me for?"},
    {'role': 'assistant', 'content': "I'm not asking you--I'm telling you. Who is on first."},
    {'role': 'user', 'content': "I'm asking you--who's on first?"},
    {'role': 'assistant', 'content': 'That\'s the man\'s name.'},
    {'role': 'user', 'content': "That's who's name?"},
    {'role': 'assistant', 'content': 'Yes.'},
]

## Connecting to the database

In [40]:
print("Creating database connections...")
# Create a table for document (chunks)
pacerDB = await PGvectorHelper.from_conn_params(
    embedding_model=e_model, 
    table_name='pacer',
    user=USER,
    password=PASSWORD,
    db_name=DB_NAME,
    host=HOST,
    port=int(PORT)
)

# Create a table for chatlogs
baseballDB = await PGvectorHelper.from_conn_params(
    embedding_model=e_model, 
    table_name='baseball',
    user=USER,
    password=PASSWORD,
    db_name=DB_NAME,
    host=HOST,
    port=int(PORT)
)
print("Connected to database.")

Creating database connections...
Connected to database.


## Create Tables

In [41]:
# Ensuring that the vector extension is installed
await pacerDB.conn.execute('''CREATE EXTENSION IF NOT EXISTS vector;''')
await baseballDB.conn.execute('''CREATE EXTENSION IF NOT EXISTS vector;''')
print("PGvector extension created and loaded.")

# Drop the table if one is found
await pacerDB.drop_table()
await baseballDB.drop_table()
print("Tables dropped.")

# Creating a new table
await pacerDB.create_doc_table()
await baseballDB.create_chatlog_table()
print("Tables created.")

await baseballDB.conn.set_type_codec(
    'JSON',
    encoder=json.dumps,
    decoder=json.loads,
    schema='pg_catalog'
)

PGvector extension created and loaded.
Tables dropped.
Tables created.


## Inserting Data

In [42]:
for index, text in enumerate(pacer_copypasta):   # For each line in the copypasta
    await pacerDB.insert_doc(                    # Insert the line into the table
        content=text,                            # The text to be embedded
        permission='public',                     # Permission metadata for access control
        title=f'Pacer Copypasta line {index}',   # Title metadata
        page_numbers=[1, 2, 3],                  # Page number metadata
        tags=['fitness', 'pacer', 'copypasta'],  # Tag metadata
    )

history_key = str(uuid.uuid4())                  # Generate a key for the chatlog
for line in abbott_and_costello:                 # For each line of dialog in the script
    await baseballDB.insert_message(             # Insert the message into the table
        history_key=history_key,                 # The key for the chatlog
        role=line['role'],
        content=line['content'],
        metadata={'genre': 'comedy', 'year': 1938}
    )

## Similarity search

In [43]:
k = 3  # Setting number of rows to return when searching

from pprint import pprint

def print_doc_results(results):  # Helper function to print document results
    print(f'RAW RETURN:')  
    pprint(results)                                                               # Print the raw results
    print(f'\nRETURNED TITLE:\n"{results[0]["title"]}"')                          # Print the title of the first result
    print(f'RETURNED CONTENT:\n"{results[0]["content"]}"')                        # Print the content of the first result
    print(f'RETURNED COSINE SIMILARITY:\n{results[0]["cosine_similarity"]:.2f}')  # Print the cosine similarity of the first result


def print_chatlog_results(results):  # Helper function to print chatlog results
    print(f'RAW RETURN:')  
    pprint(results)                                                               # Print the raw results
    print(f'\nRETURNED INDEX:\n"{results[0]["index"]}"')                          # Print the index of the first result
    print(f'RETURNED MESSAGE:\n{results[0]["role"]}: {results[0]["content"]}')    # Print the message of the first result
    print(f'RETURNED COSINE SIMILARITY:\n{results[0]["cosine_similarity"]:.2f}')  # Print the cosine similarity of the first result

### Searching the table with a perfect match:

In [44]:
search_string = '[beep] A single lap should be completed each time you hear this sound.'
print(f'Semantic Searching data using search string:\n"{search_string}"\n')

sim_search = await pacerDB.search_doc_table(
    query_string=search_string,
    limit=k
)

print_doc_results(sim_search)

Semantic Searching data using search string:
"[beep] A single lap should be completed each time you hear this sound."

RAW RETURN:
[<Record cosine_similarity=0.0 title='Pacer Copypasta line 3' content='[beep] A single lap should be completed each time you hear this sound.' permission='public' page_numbers=[1, 2, 3] tags=['fitness', 'pacer', 'copypasta']>,
 <Record cosine_similarity=0.31445924384770496 title='Pacer Copypasta line 5' content='The second time you fail to complete a lap before the sound, your test is over.' permission='public' page_numbers=[1, 2, 3] tags=['fitness', 'pacer', 'copypasta']>,
 <Record cosine_similarity=0.634082588486436 title='Pacer Copypasta line 2' content='The running speed starts slowly, but gets faster each minute after you hear this signal.' permission='public' page_numbers=[1, 2, 3] tags=['fitness', 'pacer', 'copypasta']>]

RETURNED TITLE:
"Pacer Copypasta line 3"
RETURNED CONTENT:
"[beep] A single lap should be completed each time you hear this sound.

### Searching the table with a partial match:

In [45]:
search_string = 'straight'
print(f'Semantic Searching data using search string:\n"{search_string}"\n')

sim_search = await pacerDB.search_doc_table(
    query_string=search_string,
    limit=k
)

print_results(sim_search)

Semantic Searching data using search string:
"straight"

RAW RETURN:
[<Record cosine_similarity=0.7157614573027005 title='Pacer Copypasta line 4' content='[ding] Remember to run in a straight line, and run as long as possible.' permission='public' page_numbers=[1, 2, 3] tags=['fitness', 'pacer', 'copypasta']>,
 <Record cosine_similarity=0.8959717930563745 title='Pacer Copypasta line 6' content='The test will begin on the word start. On your mark, get ready, start.' permission='public' page_numbers=[1, 2, 3] tags=['fitness', 'pacer', 'copypasta']>,
 <Record cosine_similarity=0.9200870391648666 title='Pacer Copypasta line 2' content='The running speed starts slowly, but gets faster each minute after you hear this signal.' permission='public' page_numbers=[1, 2, 3] tags=['fitness', 'pacer', 'copypasta']>]

RETURNED TITLE:
"Pacer Copypasta line 4"
RETURNED CONTENT:
"[ding] Remember to run in a straight line, and run as long as possible."
RETURNED COSINE SIMILARITY:
0.72


In [47]:
search_string = 'The guy on first base.'
print(f'Semantic Searching data using search string:\n"{search_string}"\n')

sim_search = await baseballDB.search_chatlog(
    history_key=history_key,
    query_string=search_string,
    limit=k
)

print_chatlog_results(sim_search)

Semantic Searching data using search string:
"The guy on first base."

RAW RETURN:
[<Record cosine_similarity=0.0 index=15 role=1 content='The guy on first base.' metadata_json={'genre': 'comedy', 'year': 1938}>,
 <Record cosine_similarity=0.21688772288967784 index=13 role=1 content="The fellow playin' first base." metadata_json={'genre': 'comedy', 'year': 1938}>,
 <Record cosine_similarity=0.23151054288415174 index=11 role=1 content="I mean the fellow's name on first base." metadata_json={'genre': 'comedy', 'year': 1938}>]

RETURNED INDEX:
"15"
RETURNED MESSAGE:
1: The guy on first base.
RETURNED COSINE SIMILARITY:
0.00


In [48]:
search_string = 'nickname'
print(f'Semantic Searching data using search string:\n"{search_string}"\n')

sim_search = await baseballDB.search_chatlog(
    history_key=history_key,
    query_string=search_string,
    limit=k
)

print_chatlog_results(sim_search)

Semantic Searching data using search string:
"nickname"

RAW RETURN:
[<Record cosine_similarity=0.4910637432970839 index=4 role=2 content="Nicknames, nicknames. Now, on the St. Louis team we have Who's on first, What's on second, I Don't Know is on third--" metadata_json={'genre': 'comedy', 'year': 1938}>,
 <Record cosine_similarity=0.561316881258301 index=3 role=1 content='Funny names?' metadata_json={'genre': 'comedy', 'year': 1938}>,
 <Record cosine_similarity=0.5729953094121991 index=20 role=2 content="That's the man's name." metadata_json={'genre': 'comedy', 'year': 1938}>]

RETURNED INDEX:
"4"
RETURNED MESSAGE:
2: Nicknames, nicknames. Now, on the St. Louis team we have Who's on first, What's on second, I Don't Know is on third--
RETURNED COSINE SIMILARITY:
0.49
